# 1.Import libraby and data

In [3]:
import pandas as pd
index = pd.read_csv("player_index.csv")
sta = pd.read_csv("2021-2022 Football Player Stats.csv", sep = ';', encoding = 'latin1')
print(index.shape)
print(sta.shape)

(2976, 46)
(2921, 143)


# 2.Remove duplicated records

## a.2021-2022 Football Player Stats.csv

In [4]:
player = sta['Player'].unique()
player_boolean = [True]* len(player)
sta_boolean = [0] * len(sta['Player'])
for i in range(len(sta['Player'])):
    for j in range(len(player)):
        if sta['Player'][i] == player[j] and  player_boolean[j] == True:
            player_boolean[j] = False
            sta_boolean[i] = 1

sta['bool'] = sta_boolean
sta = sta[sta['bool'] == 1]

In [6]:
sta.shape

(2786, 144)

#### => we can see that : 136 duplicated recordes removed in 2021-2022 Football Player Stats.csv

## b.player_index.csv

In [8]:
index = index.drop_duplicates(subset=["name"], keep=False)
print(index.shape)

(2974, 46)


#### => we removed 2 duplicated records 

# 3.Join 2 tables based on player's name in 2 tables

### a.Idea 2 full names taken from 2 tables are the same: if we have at least 2 similar words in any 2 full names of 2 tables, then those 2 full names are the full name of 1 player

In [9]:
# list of full names are the same in 2 tables
name_sta = []
name_index = []
for i in index['name']:
    for j in sta['Player']:
        name1 = i.strip().split(' ')
        name2 = j.strip().split(' ')
        count = 0
        for a in name1:
            if a in name2:
                count = count + 1
        if (count > 1 ):
            name_sta.append(j)
            name_index.append(i)          

In [10]:
# we remove full names in 2021-2022 Football Player Stats that don't exist in list of full names above
count_sta =[]
for i in sta['Player']:
    if i in name_sta:
        count_sta.append(name_sta.index(i))
    else:
        count_sta.append(0)
sta['count'] = count_sta
sta = sta[sta['count'] != 0 ]        

In [11]:
# check record remaning in sta data
sta.shape

(2031, 145)

In [12]:
# we remove full names in player_index.csv that don't exist in list of full names above
count_index = []
for i in index['name']:
    if i in name_index:
        count_index.append(name_index.index(i))
    else:
        count_index.append(0)
index['count'] = count_index
index = index[index['count'] != 0 ]


In [13]:
# check record remaning in index data
index.shape

(2059, 47)

### b. Select attributes of two tables that are related to player values

In [14]:
data_sta = sta[['Player', 'MP', 'Min', 'Goals', 'Assists', 'CrdY', 'CrdR','count','Comp','Shots',]]

data_index = index.copy()

### c.Combine 2 tables

In [16]:
final_data = pd.merge(data_sta, data_index, on='count')
final_data.head()

,Player,MP,Min,Goals,Assists,CrdY,CrdR,count,Comp,Shots,...,age,best_position,club,name,overall,position,potential,preferred_foot,value,wage
0,Yunis Abdelhamid,34,2983,0.06,0.00,0.15,0.03,253,Ligue 1,0.54,...,33,CB,Stade de Reims,Yunis Abdelhamid,76,LCB,76,Left,€3.6M,€23K
1,Salis Abdul Samed,31,2462,0.04,0.00,0.44,0.11,1065,Ligue 1,0.66,...,21,CDM,Clermont Foot 63,Salis Abdul Samed,70,SUB,75,Right,€2.2M,€7K
2,Laurent Abergel,34,2956,0.00,0.06,0.27,0.00,25,Ligue 1,0.91,...,28,CDM,FC Lorient,Laurent Abergel,75,CDM,75,Right,€4.9M,€18K
3,Dickson Abiama,24,726,0.00,0.12,0.37,0.00,1937,Bundesliga,2.22,...,22,ST,SpVgg Greuther Fürth,Dickson Abiama,68,SUB,76,Right,€2.7M,€9K
4,Francesco Acerbi,30,2536,0.14,0.00,0.07,0.04,941,Serie A,0.57,...,33,CB,Lazio,Francesco Acerbi,83,LCB,83,Left,€17.5M,€75K


### d.covert into csv file

In [18]:
final_data.to_csv('whole data.csv')